Now that we have cleaned the data by player names we have to clean it by year. We start by loading the datasets.

In [9]:
# We start by importing our data sets
import pandas as pd
import numpy as np

salaries = pd.read_csv("data/NBA_season1718_salary.csv")
playerData1 = pd.read_csv("data/Players_2.csv")
playerData2 = pd.read_csv("data/Seasons_Stats_2.csv")
playerData3 = pd.read_csv("data/player_data_2.csv")

salaries.head()
playerData1.head()
playerData2.head()
playerData3.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alex Abrines,2017,2018,G-F,6-6,190.0,"August 1, 1993",NaN
1,Quincy Acy,2013,2018,F,6-7,240.0,"October 6, 1990",Baylor University
2,Steven Adams,2014,2018,C,7-0,255.0,"July 20, 1993",University of Pittsburgh
3,Arron Afflalo,2008,2018,G,6-5,210.0,"October 15, 1985","University of California, Los Angeles"
4,Alexis Ajinca,2009,2017,C,7-2,248.0,"May 6, 1988",NaN


The playerData1 dataframe does not have a date column, so we will not clean it here. Seasons_Stats_2 has a column labeled "Year" so we will clean it and only leave the ones that have data from 2017 or after, as these are the years we have salaries data for. Seasons_Stats_2 has a start year and an end year column, so we will only filter for those rows that contain atleast one of the two desired years in their scope.

In [11]:
finalPlay2 = playerData2[playerData2['Year'].isin([2017, 2018])]
print(finalPlay2.shape)
finalPlay2.to_csv("data\final_Season_Stats.csv")

(506, 53)


This means that there are 506 rows that have the desired players and years in the Seasons_Stats_2 dataset. We proceed by doing a similar thing for the player_data_2.

In [10]:
finalPlay3 = playerData3[(playerData3['year_start'] <= 2018) & (playerData3['year_end'] >= 2017)]
print(finalPlay3.shape)
finalPlay3.to_csv("data\final_player_data.csv")

(423, 8)


This complicates the task of merging all the dataframes because we have two dataframes with a different a number of rows. We must figure out a way to merge necessary rows together and add unmatched rows at the end of the new dataframe.

Both of the data frames have a name and position. And the columns in the second dataframe are such that if a player had repeated rows in the first dataframe the data in the second dataframe can simply be duplicated for each repeated row.

We start by adding 4 columns: height, weight, birth_date, and college

In [13]:
finalPlay2["height"] = np.nan
finalPlay2["weight"] = np.nan
finalPlay2["birth_date"] = np.nan
finalPlay2["college"] = np.nan

finalPlay2.head()

<ipython-input-13-b2183ae3cf68>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalPlay2["height"] = np.nan
<ipython-input-13-b2183ae3cf68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalPlay2["weight"] = np.nan
<ipython-input-13-b2183ae3cf68>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,AST,STL,BLK,TOV,PF,PTS,height,weight,birth_date,college
2556,24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,...,40.0,37.0,8.0,33.0,114.0,406.0,NaN,NaN,NaN,NaN
2557,24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,...,18.0,14.0,15.0,21.0,67.0,222.0,NaN,NaN,NaN,NaN
2558,24098,2017.0,Quincy Acy,PF,26.0,DAL,6.0,0.0,48.0,-1.4,...,0.0,0.0,0.0,2.0,9.0,13.0,NaN,NaN,NaN,NaN
2559,24099,2017.0,Quincy Acy,PF,26.0,BRK,32.0,1.0,510.0,13.1,...,18.0,14.0,15.0,19.0,58.0,209.0,NaN,NaN,NaN,NaN
2560,24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,...,86.0,88.0,78.0,146.0,195.0,905.0,NaN,NaN,NaN,NaN


Now lets check for duplicate players in the second dataframe.

In [19]:
print(finalPlay3.duplicated(subset = ['name'], keep = False).sum())
print(finalPlay2.duplicated(subset = ['Player'], keep = False).sum())

0
141


This means there are no player duplicates in the second dataframe. Now we can proceed in merging these two data frames.

In [35]:
for index, row in finalPlay2.iterrows():
  player_name = row['Player']
  player_data = finalPlay3.loc[finalPlay3['name'] == player_name]
  if not player_data.empty:
    finalPlay2.at[index, 'height'] = player_data['height'].values[0]
    finalPlay2.at[index, 'weight'] = player_data['weight'].values[0]
    finalPlay2.at[index, 'birth_date'] = player_data['birth_date'].values[0]
    finalPlay2.at[index, 'college'] = player_data['college'].values[0]

finalPlay2.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,AST,STL,BLK,TOV,PF,PTS,height,weight,birth_date,college
2556,24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,...,40.0,37.0,8.0,33.0,114.0,406.0,6-6,190.0,"August 1, 1993",NaN
2557,24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,...,18.0,14.0,15.0,21.0,67.0,222.0,6-7,240.0,"October 6, 1990",Baylor University
2558,24098,2017.0,Quincy Acy,PF,26.0,DAL,6.0,0.0,48.0,-1.4,...,0.0,0.0,0.0,2.0,9.0,13.0,6-7,240.0,"October 6, 1990",Baylor University
2559,24099,2017.0,Quincy Acy,PF,26.0,BRK,32.0,1.0,510.0,13.1,...,18.0,14.0,15.0,19.0,58.0,209.0,6-7,240.0,"October 6, 1990",Baylor University
2560,24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,...,86.0,88.0,78.0,146.0,195.0,905.0,7-0,255.0,"July 20, 1993",University of Pittsburgh
